<a href="https://colab.research.google.com/github/AlcoholWolf/PY-SAVE/blob/main/CNN/ch01_11_CNN%EC%8B%A4%EC%A0%84_%EB%9E%9C%EB%93%9C%EB%A7%88%ED%81%AC%EC%9D%B4%EB%AF%B8%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ch01.11 CNN실전 - 랜드마크 이미지

* 날짜:
* 이름:

## 학습내용
    - 랜드마크 데이터를 이용한 적절한 모델을 구축 및 학습한다.

### 소개

인공지능 기반의 컴퓨터 비전 기술 및 서비스 개발에 활용하기 위해 국내 특성(지리 공간적, 기능적)이 반영된 국내 도심 민간건물, 공공기관, 관광명소, 편의시설 등 국내 도시별 주요 랜드마크 이미지 데이터 구축

### 구축목적
AI 기술 및 응용서비스 개발에 활용가치가 높은 인공지능 학습용 데이터 구축 및 개방, AI응용 서비스 개발
국내 특성(지리 공간적, 기능적)이 반영된 국내 도심 민간건물, 공공기관, 관광명소, 편의시설 등 국내 도시별 주요 랜드마크 이미지 데이터 구축

### AI허브 링크
https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=56

## 실습
---

#### **데이터로드**
```
! gdown https://drive.google.com/u/0/uc?id=19vsxVITy277RfMTmnY3qE6mH_DOZK7pt&export=download

! gdawn https://drive.google.com/u/1/uc?id=1W8a1FpXSqIQqzLKq39Tl943ykBY8OpEE&export=download
```

In [25]:
# EL-Suv : 820GB / 16Core CPU / Compect-L GPU / 32RAM / Linux17 / Inf W (300$ Limit) / 24H Space
# Colab : 80GB / None CPU / Persistence-M GPU / 12(11.24)RAM / 70W (Inf Limit) / In Space

# 모델을 .bat화 또는 Linux화 할 수 있다는 가정 하에 3배 속도, 성능

In [ ]:
! nvidia-smi

In [ ]:
! gdown https://drive.google.com/u/1/uc?id=1W8a1FpXSqIQqzLKq39Tl943ykBY8OpEE&export=download

In [ ]:
! unzip /content/jejudo.zip

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_gen = ImageDataGenerator(rescale=1.0/255.0,
                               validation_split=0.2,
                               width_shift_range=0.25,
                               height_shift_range=0.25,
                               rotation_range=180,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               vertical_flip=True,
                               brightness_range=[0.6, 1.4])

valid_gen = ImageDataGenerator(rescale=1.0/255.0,
                               validation_split=0.2)

In [18]:
IMG_SHAPE = (180, 180)
BATCH_SIZE = 8
SEED = 123
dir_path = '/content/dataset'
train_ds = train_gen.flow_from_directory(dir_path, target_size=IMG_SHAPE, batch_size=BATCH_SIZE, class_mode='categorical', seed=SEED, subset='training')
valid_ds = valid_gen.flow_from_directory(dir_path, target_size=IMG_SHAPE, batch_size=BATCH_SIZE, class_mode='categorical', seed=SEED, subset='validation')

Found 1065 images belonging to 8 classes.
Found 263 images belonging to 8 classes.


In [11]:
# import matplotlib as plt
import matplotlib.pyplot as plt

In [ ]:
# IMG : (batch_size, img_shape)
# labels : one-hot incoding useded

plt.figure(figsize=(12,8))
for imgs, labels in train_ds:
  print(imgs.shape, labels)
  for i in range(BATCH_SIZE):
    plt.subplot(2,4,i+1)
    plt.xticks([]);plt.yticks([])
    plt.imshow(imgs[i])
  break

In [20]:
base_model = tf.keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=IMG_SHAPE+(3,))

In [14]:
from tensorflow.keras import layers, models, optimizers

In [ ]:
def create_model(base_model):
  x = layers.Input(shape=IMG_SHAPE+(3,))
  y = base_model(x)
  y = layers.GlobalAveragePooling2D()(y)
  y = layers.Dropout(0.5)(y)
  y = layers.Dense(8, activation='softmax')(y)
  return models.Model(x, y)

model = create_model(base_model)
base_model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(learning_rate=0.0002), metrics=['acc'])
model.summary()

In [16]:
from tensorflow.keras import callbacks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
save_path = '/content/drive/MyDrive/eff_landmark'
TRAIN_STEP = train_ds.n//BATCH_SIZE
VALID_STEP = valid_ds.n//BATCH_SIZE
tb = callbacks.TensorBoard(log_dir=f'{save_path}/tb_log')
ckb = callbacks.ModelCheckpoint(filepath=f'{save_path}/model.h5', save_best_only=True, monitor='val_acc')
history = model.fit(train_ds, shuffle=True, epochs=5, callbacks=[tb, ckb], validation_data=valid_ds, 
                    # calq
                    steps_per_epoch=TRAIN_STEP, validation_steps=VALID_STEP)

Epoch 1/5
133/133 [==============================] - 334s 2s/step - loss: 0.4829 - acc: 0.8562 - val_loss: 2.2043 - val_acc: 0.1211
Epoch 2/5
133/133 [==============================] - 313s 2s/step - loss: 0.1318 - acc: 0.9612 - val_loss: 3.3484 - val_acc: 0.1250
Epoch 3/5
133/133 [==============================] - 299s 2s/step - loss: 0.0672 - acc: 0.9773 - val_loss: 2.1744 - val_acc: 0.2617
Epoch 4/5
133/133 [==============================] - 248s 2s/step - loss: 0.0708 - acc: 0.9801 - val_loss: 2.2119 - val_acc: 0.6250
Epoch 5/5
133/133 [==============================] - 288s 2s/step - loss: 0.0473 - acc: 0.9839 - val_loss: 0.5887 - val_acc: 0.7695
